In [1]:
from keras.datasets import mnist, cifar10
(img_train, label_train), (img_test, label_test) = cifar10.load_data()
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Input, Add, Flatten, AveragePooling2D
from keras.layers import Activation
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras import losses, optimizers

row, col = img_train.shape[1], img_train.shape[2]

if len(img_train.shape) == 4 :
    depth = img_train.shape[3]
else:
    depth = 1
    img_train = img_train.reshape(img_train.shape[0], row, col, depth)
    img_test = img_test.reshape(img_test.shape[0], row, col, depth)

img_train = img_train.astype('float32')
img_test = img_test.astype('float32')
img_train /= 255
img_test /= 255

label_train = np_utils.to_categorical(label_train, 10)
label_test = np_utils.to_categorical(label_test, 10)

class layer:
    def __init__(self, input_unit, filter_n, layer_n):
        self.result = input_unit
        self.layer_n = layer_n
        self.filter_n = filter_n
    def add(self, filter_n):
        tower_1 = Conv2D(filter_n, (3, 3), padding='same')(self.result)
        tower_1 = BatchNormalization(axis = 3)(tower_1)
        tower_1 = Activation('relu')(tower_1)
        tower_2 = Conv2D(filter_n, (3, 3), padding='same')(tower_1)
        tower_2 = BatchNormalization(axis = 3)(tower_2)
        add_1 = Add()([tower_2, self.result])
        add_1 = Activation('relu')(add_1)
        
        return add_1
    
    def implement(self, stride=2):
        tower_1 = Conv2D(self.filter_n, (3, 3), strides=stride, padding='same')(self.result)
        tower_1 = BatchNormalization(axis = 3)(tower_1)
        tower_1 = Activation('relu')(tower_1)
        tower_2 = Conv2D(self.filter_n, (3, 3), padding='same')(tower_1)
        tower_2 = BatchNormalization(axis = 3)(tower_2)
        one_by_one = Conv2D(self.filter_n, (1, 1), strides=stride, padding='same')(self.result)
        self.result = Add()([tower_2, one_by_one])
        self.result = Activation('relu')(self.result)
        
        for d in range(self.layer_n - 1):            
            self.result = self.add(self.filter_n)
            
        return self.result


Using TensorFlow backend.


Couldn't import dot_parser, loading of dot files will not be possible.


In [2]:
init_n = 16

input1 = Input(shape=(row, col, depth))
input2 = Conv2D(init_n, (3, 3), strides=1, padding='same')(input1)
input2 = BatchNormalization(axis=3)(input2)
input2 = Activation('relu')(input2)
layer1 = layer(input2, init_n, 3)
res1 = layer1.implement(1)
layer2 = layer(res1, init_n*2, 3)
res2 = layer2.implement()
layer3 = layer(res2, init_n*4, 3)
res3 = layer3.implement()
res3 = AveragePooling2D((2, 2), padding='same')(res3)
fc = Flatten()(res3)
fc = Dropout(0.2)(fc)
fc = Dense(512, activation='relu')(fc)
fc = Dropout(0.2)(fc)
fc = Dense(10, activation='softmax')(fc)
model = Model(inputs=input1, outputs=fc)
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 32, 32, 3)     0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 32, 32, 16)    448         input_1[0][0]                    
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, 32, 32, 16)    64          conv2d_1[0][0]                   
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 32, 32, 16)    0           batch_normalization_1[0][0]      
___________________________________________________________________________________________

In [3]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'

model.compile(loss=losses.categorical_crossentropy, 
              optimizer=optimizers.sgd(lr=0.01, decay=0.0001, momentum=0.9),
             metrics=['accuracy'])
model.fit(img_train, label_train, batch_size=128, epochs=100, validation_split=0.1)

score = model.evaluate(img_test, label_test)
print('\nTest score :', score[0])
print('Test accuracy :', score[1])

Train on 45000 samples, validate on 5000 samples
Epoch 1/100
45000/45000 [==============================] - 52s - loss: 1.6825 - acc: 0.3905 - val_loss: 1.6560 - val_acc: 0.4288
Epoch 2/100
45000/45000 [==============================] - 50s - loss: 1.2720 - acc: 0.5395 - val_loss: 1.1567 - val_acc: 0.5872
Epoch 3/100
45000/45000 [==============================] - 50s - loss: 1.0709 - acc: 0.6146 - val_loss: 1.2293 - val_acc: 0.5728
Epoch 4/100
45000/45000 [==============================] - 50s - loss: 0.9400 - acc: 0.6644 - val_loss: 1.0774 - val_acc: 0.6268
Epoch 5/100
45000/45000 [==============================] - 50s - loss: 0.8327 - acc: 0.7029 - val_loss: 1.1668 - val_acc: 0.6182
Epoch 6/100
45000/45000 [==============================] - 49s - loss: 0.7480 - acc: 0.7329 - val_loss: 0.8354 - val_acc: 0.7060
Epoch 7/100
45000/45000 [==============================] - 49s - loss: 0.6811 - acc: 0.7585 - val_loss: 0.9468 - val_acc: 0.6958
Epoch 8/100
45000/45000 [=======================

In [3]:
score = model.evaluate(img_test, label_test)
print('\nTest score :', score[0])
print('Test accuracy :', score[1])

RuntimeError: You must compile a model before training/testing. Use `model.compile(optimizer, loss)`.